In [25]:
import os
import pandas as pd

In [26]:
os.chdir("/Users/lajoyce/Documents/genomics/epiage/BayesAge")

##### The methylation calling function from BSBolt can be used to generate the DNA methylation matrices. Futhermore, the training matrix must have a target column named "Age".
Once the matrix is generated, you can proceed to the analysis below. For example purposes, we are going to take 10 random samples out of the 458 methylation matrix for our cross validation.

In [27]:
full_DNAm_df = pd.read_csv("/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/sample_458_age.csv", sep="\t", low_memory=False)

In [28]:
full_DNAm_df.tail(10)

,sampleID,chr1_100424052,chr1_100424116,chr1_100718749,chr1_100718777,chr1_100719693,chr1_100719742,chr1_100719744,chr1_100719756,chr1_100719847,...,chrX_70133602,chrX_70133614,chrX_72573198,chrX_72573200,chrX_72573216,chrX_91434796,chrX_91434801,chrX_91434803,chrX_91434826,Age
448,Turk-C26,0.009901,0.006494,0.015038,0.210526,0.396226,0.800000,0.549296,0.451456,0.492147,...,0.000000,0.011765,0.006757,0.006289,0.015957,0.000000,0.000000,0.011628,0.036585,23.0
449,Turk-C28,0.064516,0.005025,0.051429,0.305085,0.418239,0.845890,0.654237,0.545455,0.450237,...,0.415094,0.323529,0.292035,0.348571,0.121896,0.040404,0.015075,0.030151,0.031579,24.0
450,Turk-C31,0.014925,0.000000,0.112500,0.368421,0.383648,0.826087,0.627329,0.483871,0.432624,...,0.455357,0.345794,0.322835,0.504000,0.147436,0.078652,0.055556,0.088889,0.093023,24.0
451,Turk-C32,0.098901,0.006803,0.036810,0.210884,0.262222,0.630332,0.490385,0.283654,0.309677,...,0.435644,0.308081,0.268293,0.319249,0.131783,0.032680,0.032258,0.064516,0.013245,23.0
452,Turk-C33,0.029630,0.008658,0.024752,0.232323,0.420455,0.780556,0.663014,0.555866,0.543253,...,0.485612,0.399267,0.356796,0.355450,0.177866,0.048246,0.105727,0.118421,0.080357,40.0
453,Turk-C34,0.026490,0.012766,0.098446,0.275862,0.278075,0.652422,0.501425,0.394203,0.440972,...,0.429224,0.373206,0.197222,0.313187,0.181159,0.052133,0.023697,0.056604,0.043062,24.0
454,Turk-C37,0.122093,0.015385,0.041667,0.225664,0.371230,0.712159,0.542998,0.419192,0.437126,...,0.000000,0.000000,0.004926,0.014493,0.008511,0.009259,0.018519,0.018519,0.019231,80.0
455,Turk-C39,0.008065,0.009852,0.030435,0.209756,0.173913,0.632836,0.447447,0.260450,0.310580,...,0.453901,0.443636,0.310526,0.341085,0.152655,0.072398,0.141593,0.074890,0.079070,19.0
456,Turk-C40,0.016000,0.010101,0.017544,0.203488,0.181818,0.563707,0.414062,0.269076,0.339806,...,0.455556,0.440000,0.195364,0.321543,0.121827,0.175676,0.033557,0.100671,0.103448,31.0
457,Turk-C42,0.027778,0.000000,0.049180,0.409836,0.393939,0.714286,0.650000,0.474747,0.386364,...,0.040000,0.000000,0.022222,0.000000,0.019231,0.027778,0.000000,0.000000,0.000000,35.0


In [31]:
files_to_test = os.listdir("/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/CGmap")

In [33]:
files_to_test.remove(".DS_Store")

In [34]:
files_to_test

['SalivaDiabetes_20038.CGmap',
 'PFitness_44.CGmap',
 'P41019081704087.CGmap',
 'SalivaDiabetes_20000.CGmap',
 '140279.CGmap',
 '130010.CGmap',
 'Turk-60.CGmap',
 'Turk-5.CGmap',
 'Neonate1036.CGmap',
 'PFitness_1.CGmap']

In [36]:
def remove_extension(file_list):
    # Create a new list to store file names without extensions
    files_without_extension = []

    # Iterate through each file in the list
    for file_name in file_list:
        # Use os.path.splitext to split the file name and extension
        file_name_without_extension, _ = os.path.splitext(file_name)

        # Add the file name without extension to the new list
        files_without_extension.append(file_name_without_extension)

    return files_without_extension

In [37]:
files_without_extension = remove_extension(files_to_test)
print(files_without_extension)

['SalivaDiabetes_20038', 'PFitness_44', 'P41019081704087', 'SalivaDiabetes_20000', '140279', '130010', 'Turk-60', 'Turk-5', 'Neonate1036', 'PFitness_1']


In [38]:
reduced_DNAm_matrix = full_DNAm_df[~full_DNAm_df['sampleID'].isin(files_without_extension)]

In [40]:
reduced_DNAm_matrix.shape

(448, 46519)

In [44]:
reduced_DNAm_matrix.to_csv("/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/reduced_DNAm_matrix.csv", sep="\t", index=False)

# Construct Reference

In [45]:
from BayesAge import construct_reference

In [46]:
df = pd.read_csv("/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/reduced_DNAm_matrix.csv", sep="\t", low_memory=False)

df.head(5)

,sampleID,chr1_100424052,chr1_100424116,chr1_100718749,chr1_100718777,chr1_100719693,chr1_100719742,chr1_100719744,chr1_100719756,chr1_100719847,...,chrX_70133602,chrX_70133614,chrX_72573198,chrX_72573200,chrX_72573216,chrX_91434796,chrX_91434801,chrX_91434803,chrX_91434826,Age
0,P41019081704090,0.010256,0.000000,0.000000,0.235808,0.460432,0.796334,0.675676,0.520737,0.512941,...,0.442708,0.423913,0.216117,0.282609,0.081356,0.189944,0.182796,0.126316,0.225806,46.0
1,P41019081704100,0.325843,0.010204,0.038961,0.226776,0.682594,0.790660,0.724473,0.550918,0.646091,...,0.423963,0.228856,0.211356,0.389571,0.167082,0.127072,0.237838,0.088083,0.148352,47.0
2,P41019081704106,0.117647,0.034274,0.054332,0.157618,0.399802,0.780488,0.532505,0.465587,0.429896,...,0.326284,0.355224,0.244514,0.376582,0.181440,0.132000,0.083019,0.136364,0.094862,38.0
3,P41019081704108,0.045213,0.006135,0.032438,0.197802,0.350000,0.743935,0.582768,0.435679,0.396048,...,0.418874,0.344068,0.246377,0.378438,0.180915,0.078947,0.080235,0.084778,0.058601,47.0
4,P41019081704109,0.077869,0.005376,0.084189,0.350495,0.380319,0.770186,0.668742,0.567183,0.488789,...,0.409231,0.384365,0.180666,0.339062,0.113208,0.027119,0.061489,0.056604,0.039394,47.0


In [47]:
construct_reference(training_matrix="/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/reduced_DNAm_matrix.csv",
reference_name = "reference_model",
output_path = "/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/output_files/",
zero_met_replacement=0.001,
one_met_replacement=0.999,
min_age = 0,
max_age = 100,
age_step = 1,
tau = 0.7)


Reference model dataset written to '/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/output_files/'
Report file generated at '/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/output_files/.report.txt'
-----------------------------------------------------



Time to run construct reference: 398.553 seconds

 The reference is constructed and saved!


,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,spearman_rank,p_value
ChrPos,,,,,,,,,,,,,,,,,,,,,
chr1_100424052,0.100447,0.105414,0.110405,0.115418,0.120450,0.125501,0.130567,0.135647,0.140736,0.145833,...,0.129177,0.117018,0.104830,0.092616,0.080379,0.068122,0.055848,0.043560,0.149929,0.001460
chr1_100424116,0.020291,0.020925,0.021565,0.022209,0.022858,0.023511,0.024168,0.024829,0.025493,0.026159,...,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.077685,0.100556
chr1_100718749,0.033153,0.034519,0.035882,0.037242,0.038599,0.039953,0.041304,0.042652,0.043996,0.045338,...,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,-0.045657,0.334951
chr1_100718777,0.209831,0.211504,0.213162,0.214804,0.216432,0.218046,0.219647,0.221236,0.222813,0.224381,...,0.148821,0.144996,0.141205,0.137449,0.133725,0.130031,0.126365,0.122728,-0.070481,0.136357
chr1_100719693,0.284982,0.291758,0.298525,0.305282,0.312030,0.318768,0.325496,0.332213,0.338919,0.345614,...,0.314510,0.304811,0.295076,0.285310,0.275512,0.265687,0.255834,0.245957,0.001980,0.966672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_72573216,0.054403,0.055405,0.056391,0.057363,0.058320,0.059264,0.060194,0.061113,0.062021,0.062918,...,0.086038,0.085840,0.085622,0.085387,0.085135,0.084868,0.084585,0.084289,0.059140,0.211533
chrX_91434796,0.028428,0.029749,0.031063,0.032370,0.033672,0.034968,0.036257,0.037542,0.038822,0.040097,...,0.078243,0.077716,0.077204,0.076706,0.076222,0.075751,0.075294,0.074850,0.174538,0.000205
chrX_91434801,0.021164,0.022274,0.023382,0.024486,0.025588,0.026687,0.027783,0.028876,0.029967,0.031056,...,0.080114,0.080438,0.080779,0.081139,0.081514,0.081906,0.082312,0.082733,0.177436,0.000160


# Sample Files Preparation

In [48]:
from BayesAge import process_cgmap_file

In [50]:
process_cgmap_file(cgmap_directory="/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/CG_map_try2/",
output_path = "/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/processed_samples/",
n_cores = 1,
split = ".",
chunksize = 1)

----------------------------------------------------------
Loading .cgmap files from '/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/CG_map_try2/'
Number of BSBolt .CGmap files = 10
First .CGmap file name: '130010.CGmap'
----------------------------------------------------------

----------------------------------------------------------
Starting parallel loading and processing of .CGmap files...


sample loading progress :   0%|          | 0/10 [00:00<?, ? sample methylomes/s]


Parallel loading complete!
Processed matrices written to '/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/processed_samples/'
----------------------------------------------------------

Time elapsed to process CGmap files = 12.835 seconds

process_cgmap run complete!


# Age Prediction

In [53]:
from BayesAge import bdAge
import os
import pandas as pd

In [54]:
cgmap_samples = os.listdir("/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/processed_samples")

In [55]:
bdAge(sample_dir_or_dict="/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/processed_samples/",
    sample_set_name="test",
    reference_data="/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/output_files/reference_model",
    output_path="/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/predictions/",
    selection_mode="numCpGs",
    CpG_parameter=8,
    min_age=0,
    max_age=100,
    age_step=1,
    n_cores = 1,
    chunksize = 5)

bdAge algorithm starting!

----------------------------------------
Profiling epigenetic age in 'test' 
Loading processed CGmap methylation files from '/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/processed_samples/'...
Number of samples to analyze: 10

bdAge parameters: 
---------------------------------------------------------
Using reference training data: reference_model
Shape of reference matrix: 46,517 CpGs, 103 metric columns


Using 1 cores with chunksize of 5


Setting minimum age to 0 month(s)/year(s)
Setting maximum age to 100 month(s)/year(s)
Using age step of 1 month(s)/year(s)


Using profiling mode: numCpGs
---> Profiling top 8 age-related CpGs by absolute Spearman rank
---------------------------------------------------------------------


----------------------------------------------------------
Starting parallel processing of all samples with 1 cores!



bdAge progress :   0%|          | 0/10 [00:00<?, ? Age predictions/s]




Predictions stored in '/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/predictions/'
----------------------------------------------------------

Time elapsed to generate bdAge results = 5.281 seconds

BdAge run complete!


# Prediction Analysis

In [56]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

In [57]:
prediction_df = pd.read_csv("/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/predictions/test-train(reference_model)-mode(numCpGs)-param(8CpGs).tsv", sep="\t")

In [58]:
prediction_df.head(5)

,Sample,Predicted Age,Mean Meth Count,Mean Coverage,Sample Coverage,Intersections,Ages Tested,Age Likelihood,Selected CpGs,Number CpGs,Correlations,Methylation Count,Coverage Count
0,130010,60.0,2.236188,40.826607,532220,43155,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[-358.6887484540738, -320.9842852566147, -291....","['chr7_8442747', 'chr5_16179182', 'chr19_18233...",8,"[0.7664719975405361, 0.7590120982722924, 0.750...","[26, 34, 15, 33, 15, 19, 14, 23]","[64, 55, 59, 58, 59, 49, 55, 61]"
1,140279,73.0,1.813340,36.253441,643396,43163,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[-321.59439587392615, -292.23135041815806, -26...","['chr7_8442747', 'chr5_16179182', 'chr19_18233...",8,"[0.7664719975405361, 0.7590120982722924, 0.750...","[27, 22, 10, 29, 8, 12, 10, 24]","[38, 38, 39, 40, 39, 37, 32, 40]"
2,Neonate1036,0.0,1.734403,40.047641,606053,43163,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[-12.420153126334839, -14.313524696452983, -16...","['chr7_8442747', 'chr5_16179182', 'chr19_18233...",8,"[0.7664719975405361, 0.7590120982722924, 0.750...","[0, 1, 0, 0, 1, 1, 2, 1]","[52, 62, 46, 63, 46, 59, 47, 48]"
3,P41019081704087,44.0,2.003780,44.460925,641484,42891,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[-194.6710225735422, -172.3996594456457, -155....","['chr7_8442747', 'chr5_16179182', 'chr19_18233...",8,"[0.7664719975405361, 0.7590120982722924, 0.750...","[23, 13, 11, 11, 17, 9, 10, 16]","[54, 31, 64, 33, 63, 31, 37, 62]"
4,PFitness_1,36.0,1.944717,38.199254,683443,43163,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[-139.44845370318987, -123.78063299357507, -11...","['chr7_8442747', 'chr5_16179182', 'chr19_18233...",8,"[0.7664719975405361, 0.7590120982722924, 0.750...","[19, 19, 9, 9, 4, 7, 6, 11]","[49, 29, 50, 29, 50, 29, 44, 49]"


In [59]:
subsample_predicted_ages = prediction_df[["Sample", "Predicted Age"]]

In [60]:
subsample_predicted_ages.shape

(10, 2)

In [61]:
samples_ = prediction_df["Sample"].tolist()

print(samples_)

['130010', '140279', 'Neonate1036', 'P41019081704087', 'PFitness_1', 'PFitness_44', 'SalivaDiabetes_20000', 'SalivaDiabetes_20038', 'Turk-5', 'Turk-60']


In [62]:
true_ages = pd.read_csv("/Users/lajoyce/Documents/genomics/epiage/BayesAge/examples/sample_458_age.csv", sep="\t", low_memory=False)

true_ages.head(5)

,sampleID,chr1_100424052,chr1_100424116,chr1_100718749,chr1_100718777,chr1_100719693,chr1_100719742,chr1_100719744,chr1_100719756,chr1_100719847,...,chrX_70133602,chrX_70133614,chrX_72573198,chrX_72573200,chrX_72573216,chrX_91434796,chrX_91434801,chrX_91434803,chrX_91434826,Age
0,P41019081704087,0.093842,0.026515,0.006906,0.174535,0.319149,0.697318,0.483591,0.336207,0.306792,...,0.003676,0.000000,0.002294,0.008830,0.001996,0.000000,0.018405,0.011905,0.045161,34.0
1,P41019081704090,0.010256,0.000000,0.000000,0.235808,0.460432,0.796334,0.675676,0.520737,0.512941,...,0.442708,0.423913,0.216117,0.282609,0.081356,0.189944,0.182796,0.126316,0.225806,46.0
2,P41019081704100,0.325843,0.010204,0.038961,0.226776,0.682594,0.790660,0.724473,0.550918,0.646091,...,0.423963,0.228856,0.211356,0.389571,0.167082,0.127072,0.237838,0.088083,0.148352,47.0
3,P41019081704106,0.117647,0.034274,0.054332,0.157618,0.399802,0.780488,0.532505,0.465587,0.429896,...,0.326284,0.355224,0.244514,0.376582,0.181440,0.132000,0.083019,0.136364,0.094862,38.0
4,P41019081704108,0.045213,0.006135,0.032438,0.197802,0.350000,0.743935,0.582768,0.435679,0.396048,...,0.418874,0.344068,0.246377,0.378438,0.180915,0.078947,0.080235,0.084778,0.058601,47.0


In [63]:
merged_df = pd.merge(subsample_predicted_ages, true_ages, left_on="Sample", right_on="sampleID", how="inner")

In [64]:
mini_merged_df = merged_df[["Sample", "Predicted Age", "Age"]]

In [65]:
mini_merged_df.head(3)

,Sample,Predicted Age,Age
0,130010,60.0,68.09
1,140279,73.0,67.48
2,Neonate1036,0.0,0.00


In [67]:
x = mini_merged_df["Age"]
y = mini_merged_df["Predicted Age"]

model = sm.OLS(y, sm.add_constant(x))

results = model.fit()
print(results.params)
print(results.summary())

const    2.756752
Age      0.954315
dtype: float64
                            OLS Regression Results                            
Dep. Variable:          Predicted Age   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     55.79
Date:                Sat, 09 Dec 2023   Prob (F-statistic):           7.13e-05
Time:                        20:59:04   Log-Likelihood:                -34.506
No. Observations:                  10   AIC:                             73.01
Df Residuals:                       8   BIC:                             73.62
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c

In [68]:
r_squared = results.rsquared

b, m = results.params

print(b, m)

2.7567523484409624 0.9543151175590529


In [71]:
actual = mini_merged_df['Age'].tolist()
predicted = mini_merged_df['Predicted Age'].tolist()

n = 10
sum = 0
for i in range(n):
    sum += abs(actual[i] - predicted[i])

error = sum/n

print("Mean absolute error: " + str(error))

Mean absolute error: 5.961
